<a href="https://colab.research.google.com/github/kdwang1808/2020-MCM/blob/master/Text_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
import sys
os.chdir('drive/Colab Notebooks/2020MCM_PC')

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm as tqdm
from nltk.corpus import stopwords
stop = stopwords.words("english")
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torchsummary

In [0]:
hair = pd.read_csv('hair.csv')
micro = pd.read_csv('micro.csv')
baby = pd.read_csv('baby.csv')

In [8]:
hair.head(8)

,yy,mm,dd,product_parent,star_rating,helpful_votes,total_votes,review_headline,review_body
0,2008,1,1,201,5,7,8,Simply The Best Blow Dryer on The Market,This is the best standard blow dryer on the ma...
1,2008,1,7,165,5,4,6,My new best friend!,With a four year warranty I had to try it and ...
2,2008,1,8,75,5,16,17,"Terrific hair dryer, with caveat",Works great; same dryer used by my hair stylis...
3,2008,1,10,75,5,2,2,GREAT DRYER,This has cut down on my drying time and is ver...
4,2008,1,11,241,5,40,42,Conair 209R Ion Shine(s) for Under $20.00!,You can't get much for $20 these days. Howeve...
5,2008,1,12,241,1,11,14,Cord Keeper Breaks!,"I researched all the hairdryers on amazon, and..."
6,2008,1,13,180,4,5,6,Hard to believe it could be so good,I bought this dryer based on the user reviews ...
7,2008,1,28,244,4,0,0,conair hairdryer,It is okay. It doesn't get hot enough and the ...


In [0]:
def cleaning( review, remove_stopwords=True):
  review_text = re.sub("[^a-zA-Z]"," ", review)
  words = review_text.lower().split()
  if remove_stopwords:
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    b=[]
    stemmer = english_stemmer 
    for word in words:
        b.append(stemmer.stem(word))
    return(b)

In [0]:
hair_body = []
for review in hair['review_body']:
  review = str(review)
  hair_body.append( " ".join(cleaning(review)))
    
hair_headline = []
for review in hair['review_headline']:
  review = str(review)
  hair_headline.append( " ".join(cleaning(review)))

micro_body = []
for review in micro['review_body']:
  review = str(review)
  micro_body.append( " ".join(cleaning(review)))
    
micro_headline = []
for review in micro['review_headline']:
  review = str(review)
  micro_headline.append( " ".join(cleaning(review)))

baby_body = []
for review in baby['review_body']:
  review = str(review)
  baby_body.append( " ".join(cleaning(review)))
    
baby_headline = []
for review in baby['review_headline']:
  review = str(review)
  baby_headline.append( " ".join(cleaning(review)))

In [11]:
micro_body

['surpris amount size food oven accomod cook everyth meatloaf pound cake even lock juic bought sharp rang microwav gave eleven year constant use use sharp featur made logic choic time want replac convent oven also yrs old never bake even sharp micro convect combo longer replac convent rang oven use storag',
 'price think beat sharp microwav want basic microwav job replac countertop micro rate watt concern rate watt sharp would inadequ fear groundless sharp function everi bit fast older one like sleek appear without protrud door handl recommend two peopl mount oven howev sinc help avail abl easili mount littl thought rate anyth would mount instruct confus coupl area main vent info good buy',
 'much research settl terrif microwav everi brand microwav seem malfunct help horror stori end chanc microwav one seem better work splendid without problem nois mechan part experi sharp carousel star home work depend effici good look inexpens cubic foot carousel commodi enough larg casserol design k

In [0]:
Top_hair = pd.Series(' '.join(hair_body).lower().split()).value_counts()[:10]
print("Top hair body\n", Top_hair)

Top hair body
 hair     9199
dryer    8893
use      4305
one      3631
dri      3367
work     2862
great    2755
br       2655
love     2381
like     2194
dtype: int64


In [0]:
Top_hair_headline = pd.Series(' '.join(hair_headline).lower().split()).value_counts()[:10]
print("Top hair headline\n", Top_hair_headline)

Top hair headline
 dryer      2030
star       1775
great      1494
hair       1456
five       1226
good        710
love        687
work        495
product     441
best        316
dtype: int64


In [16]:
micro["review_body"][1]

"For the price I don't think you can beat this Sharp microwave.  I just wanted a basic microwave that did the job.  I was replacing a countertop micro that was rated at 1200 watts and I was concerned that the rated 950 watts of the Sharp would be inadequate.  My fears were groundless as the Sharp functions every bit as fast the older one.  I like the sleek appearance without a protruding door handle.  They recommend two people to mount this oven, however since I didn't have help available I was able to easily mount it myself with a little thought.  If I were to rate it down for anything it would be that the mounting instructions are confusing in a couple of areas, mainly in the venting info.  All-in-all a good buy."